# PyTerrier Notebook for Full-Rank Submissions

This notebook serves as a baseline full-rank submission for [TIRA](https://tira.io)/[TIREx](https://tira.io/tirex) that builds a PyTerrier index and subsequently creates a run with BM25.

### Step 1: Ensure Libraries are Imported

In [1]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88 ir_datasets
else:
    print('We are in the TIRA sandbox.')

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt


/Users/dominicwild/anaconda3/envs/IR/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Load the data

In [3]:
data = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/validation-20231104-training')

Load ir_dataset "ir-lab-jena-leipzig-wise-2023/validation-20231104-training" from tira.


In [4]:
print('See the first two queries:')
topics = data.get_topics('title')
print(topics.head(2))

See the first two queries:
No settings given in /Users/dominicwild/.tira/.tira-settings.json. I will use defaults.
No settings given in /Users/dominicwild/.tira/.tira-settings.json. I will use defaults.
       qid              query
0  q072224     purchase money
1  q072226  purchase used car


### Step 3: Build the Index

In [5]:
print('Build index:')
iter_indexer = pt.IterDictIndexer("/tmp/index", meta={'docno': 100}, verbose=True)
!rm -Rf /tmp/index
indexref = iter_indexer.index(data.get_corpus_iter())

print('Done. Index is created')

Build index:
No settings given in /Users/dominicwild/.tira/.tira-settings.json. I will use defaults.
No settings given in /Users/dominicwild/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents:   0%|          | 0/61307 [00:00<?, ?it/s]

No settings given in /Users/dominicwild/.tira/.tira-settings.json. I will use defaults.
No settings given in /Users/dominicwild/.tira/.tira-settings.json. I will use defaults.


ir-lab-jena-leipzig-wise-2023/validation-20231104-training documents: 100%|██████████| 61307/61307 [00:25<00:00, 2419.25it/s]


Done. Index is created


### Step 4: Create the Retrieval Pipeline

In [6]:
import numpy as np

tf_idf = pt.BatchRetrieve(indexref, wmodel="TF_IDF", controls={"tf_idf.b" : 1})
klq = pt.rewrite.KLQueryExpansion(indexref, verbose=True)
tfidf_klq_tuned = tf_idf >> klq >> tf_idf

### Step 5: Create the Run and Persist the Run

In [7]:
print('Create run')
run = tfidf_klq_tuned(topics)
print('Done, run was created')

Create run


Transformer: 100%|██████████| 878/878 [00:18<00:00, 47.36q/s]


Done, run was created


In [8]:
persist_and_normalize_run(run, 'tfidf_klq_tuned')

I use the environment variable "TIRA_OUTPUT_DIR" to determine where I should store the run file using "." as default.
Done. run file is stored under "./run.txt".
